<a href="https://colab.research.google.com/github/gosantam/Tablas/blob/main/Conversores/coord2table_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transforma imágenes de tablas a código latex

# $jpg \rightarrow xlsx \rightarrow latex$

- **Principal problema:** hay que utilizar un programa externo a python para realizar la conversión.

## Librerías necesarias

In [ ]:
%%capture
pip install --user https://github.com/pyexcel/pyexcel-xlsx/archive/master.zip

In [ ]:
from pyexcel_xlsx import get_data
import numpy as np
import pandas as pd
import datetime

## Paso 1
Se puede transformar la **tabla.jgp** (o cualquier formato de imagen) a el excel **tabla.xlsx** en el siguiente enlace:

- https://online2pdf.com/en/convert-jpg-to-excel

De hecho, puedes añadir varias imágenes a la vez y nos devolverá un xlsx con todas las tablas.



```
# Copiamos la ruta del archivo xlsx creado. En google colab sería el propio: tabla.xlsx
```



## Paso 2
Utilizando la libreria **pyexcel_xlsx**, en concreto el método **get_data**, extraemos todas las tablas del archivo xlsx

## Con varias tablas

In [ ]:
rows = get_data('/content/us-004-2-1 (2 files merged).xlsx')
rows = dict(rows)
print(rows)

{'Table 1': [['Loan type', 'Loan type', datetime.datetime(2009, 12, 31, 0, 0), datetime.datetime(2009, 12, 31, 0, 0), datetime.datetime(2009, 12, 31, 0, 0), '', datetime.datetime(2010, 12, 31, 0, 0), datetime.datetime(2010, 12, 31, 0, 0), datetime.datetime(2010, 12, 31, 0, 0), '', datetime.datetime(2011, 6, 30, 0, 0), datetime.datetime(2011, 6, 30, 0, 0)], ['Loan type', 'Loan type', "$000's", "$000's", 'O/o', "$000's", "$000's", "$000's", 'O/o', "$000's", "$000's", 'O/o'], ['Realestate loans', 'Realestate loans'], ['1-4 family residential mortgage', '1-4 family residential mortgage', '4,151,000', '4,151,000', 250, '4,090,000', '4,090,000', '4,090,000', 275, 'II\'"', '3,925,000', 249], ['Commercial Mortgage', 'Commercial Mortgage', 361, 361, 22, 331, 331, 331, 22, 284, 284, 18], ['Multifamily residential (5 or more)', 'Multifamily residential (5 or more)', 380, 380, 23, 327, 327, 327, 22, 327, 327, 21], ['Construction Loans', 'Construction Loans', 173, 173, 10, 148, 148, 148, 10, 170, 1

In [ ]:
rows.keys() #  En el excel hay dos tablas, pero no sabe separarlas en dos.
# También podemos indicarle que nos devuelva dos xlsx por seprado. Lo cual puede ser más útil.

dict_keys(['Table 1'])

### Tabla 1

In [ ]:
print(rows['Table 1'])

[['Loan type', 'Loan type', datetime.datetime(2009, 12, 31, 0, 0), datetime.datetime(2009, 12, 31, 0, 0), datetime.datetime(2009, 12, 31, 0, 0), '', datetime.datetime(2010, 12, 31, 0, 0), datetime.datetime(2010, 12, 31, 0, 0), datetime.datetime(2010, 12, 31, 0, 0), '', datetime.datetime(2011, 6, 30, 0, 0), datetime.datetime(2011, 6, 30, 0, 0)], ['Loan type', 'Loan type', "$000's", "$000's", 'O/o', "$000's", "$000's", "$000's", 'O/o', "$000's", "$000's", 'O/o'], ['Realestate loans', 'Realestate loans'], ['1-4 family residential mortgage', '1-4 family residential mortgage', '4,151,000', '4,151,000', 250, '4,090,000', '4,090,000', '4,090,000', 275, 'II\'"', '3,925,000', 249], ['Commercial Mortgage', 'Commercial Mortgage', 361, 361, 22, 331, 331, 331, 22, 284, 284, 18], ['Multifamily residential (5 or more)', 'Multifamily residential (5 or more)', 380, 380, 23, 327, 327, 327, 22, 327, 327, 21], ['Construction Loans', 'Construction Loans', 173, 173, 10, 148, 148, 148, 10, 170, 170, 11], ['C

### Tabla 2

In [ ]:
# print(rows['Table 2']) // Da error

## Con una tabla

In [ ]:
rows = get_data('/content/us-040-2-1.xlsx')
rows = dict(rows)
print(rows)

{'Table 1': [['Species', 'Wildlife Criterion (pg/L)', 'Wildlife Criterion (pg/L)', 'Wildlife Criterion (pg/L)'], ['Species', 'GLWQI', 'GLWQI', 'Mercury Study Report to Congress'], ['Mink', 2880, 2880, 1038], ['Otter', 1930, 1930, 764], ['Kingfisher', 1040, 1040, 598], ['Osprey', 'Not done', 'Not done', 1498], ['Eagle', 1920, 1920, 1818]]}


In [ ]:
print(rows['Table 1'])

[['Species', 'Wildlife Criterion (pg/L)', 'Wildlife Criterion (pg/L)', 'Wildlife Criterion (pg/L)'], ['Species', 'GLWQI', 'GLWQI', 'Mercury Study Report to Congress'], ['Mink', 2880, 2880, 1038], ['Otter', 1930, 1930, 764], ['Kingfisher', 1040, 1040, 598], ['Osprey', 'Not done', 'Not done', 1498], ['Eagle', 1920, 1920, 1818]]


## Paso 3
Realizamos la conversión a latex. Para ello hay que pocesar las filas y columnas de la matriz que nos devuelve el método **get_data**

### Implementación de funciones

In [ ]:
def borrarColumnas(rows):
  c = rows[:,1:]==rows[:,:-1]
  c = np.array([[False]+list(f) for f in c])
  a = np.array([True for i in range(0,rows.shape[1])])
  for f in c:
    a = a*f
  rows = rows.transpose()
  rows_a = []
  for i in range(0,len(rows)):
    if a[i] == False:
      rows_a.append(list(rows[i]))
  return np.array(rows_a).transpose()

In [ ]:
def borrarFilas(rows):
  c = rows[1:,:]==rows[:-1,:]
  c = np.array([[False for i in range(0,rows.shape[1])]]+[list(f) for f in c])
  a = np.array([True for i in range(0,rows.shape[0])])
  for f in c.transpose():
    a = a*f
  rows_a = []
  for i in range(0,len(rows)):
    if a[i] == False:
      rows_a.append(list(rows[i]))
  return np.array(rows_a)

In [ ]:
def xlsx2matriz(path):
  rows = get_data(path)
  rows = dict(rows)
  rows = rows['Table 1']
  M = max(len(f) for f in rows)
  for i in range(0,len(rows)):
    if len(rows[i])<M: rows[i]=rows[i]+[' ' for j in range(0,M-len(rows[i]))]
  rows = np.array(rows)
  rows = borrarFilas(rows)
  rows = borrarColumnas(rows)
  for i in range(0,len(rows)):
    for j in range(0,len(rows[i])):
      if type(rows[i][j]) == datetime.datetime: rows[i][j] = rows[i][j].strftime("%D")
      if type(rows[i][j]) == int: rows[i][j] = str(rows[i][j])
      rows[i][j] = rows[i][j].replace('$','\$').replace('&','and').replace('%','\%')
  return [list(f) for f in rows]

In [ ]:
def matriz2latex(rows):
  c = ''
  print('\\begin{table}')
  c = c+'\\begin{table} \n'
  print('\\begin{center}')
  c = c+'\\begin{center} \n'
  print('\\begin{tabular}{|'+'|'.join(['c' for i in range(0,max([len(f) for f in rows]))])+'|}')
  c = c+'\\begin{tabular}{|'+'|'.join(['c' for i in range(0,max([len(f) for f in rows]))])+'|} \n'

  anterior = rows[0][0]
  k1,k2 = 0,0
  for i in range(0,len(rows)-1):
    cell = rows[i+1][0]
    if cell == anterior:
      k2+=1
    else:
      if k2>k1:
        rows[k1][0] = '\multirow{'+str(k2-k1+1)+'}{*}{'+anterior+'}'
        for j in range(k1+1,k2+1): rows[j][0]=''
      k2+=1
      k1=k2
      anterior = cell
  if k2>k1:
    rows[k1][0] = '\multirow{'+str(k2-k1+1)+'}{*}{'+anterior+'}'
    for j in range(k1+1,k2+1): rows[j][0]=''
  
  for i in range(0,len(rows)):
    # Tratando las multi-columnas
    anterior = rows[i][0]
    k1,k2,j=0,0,0
    for cell in rows[i][1:]:
      if anterior == cell:
        k2+=1
      else:
        if k2 > k1:
          rows[i][rows[i].index(anterior)] = '\multicolumn{'+str(k2-k1+1)+'}{|c|}{'+anterior+'}'
          for r in range(0,k2-k1):
            rows[i].remove(anterior)
        anterior = cell
        k1,k2=0,0
    if k2 > k1:
      elementos[anterior] = (k1,k2)
      rows[i][rows[i].index(anterior)] = '\multicolumn{'+str(k2-k1+1)+'}{|c|}{'+anterior+'}'
      for r in range(0,k2-k1):
        rows[i].remove(anterior)
    if rows[i][0] == '':
      print('\\cline{2-'+str(max([len(f) for f in rows]))+'}')
      c = c+'\\cline{2-'+str(max([len(f) for f in rows]))+'} \n'
    else:
      print('\\hline')
      c = c+'\\hline \n'
    print(' & '.join(rows[i])+' \\\\')
    c = c+' & '.join(rows[i])+' \\\\ \n'  
  print('\\hline')
  c = c+'\\hline \n'
  print('\\end{tabular}')
  c=c+'\\end{tabular} \n'
  print('\\end{center}')
  c=c+'\\end{center} \n'
  print('\\end{table}')
  c=c+'\\end{table} \n'
  return c

In [ ]:
def xlsx2latex(path):
  rows = xlsx2matriz(path)
  c = matriz2latex(rows)
  a = path[:path.find('.xlsx')]+'.txt'
  with open(a,'w') as f:
    f.write(c)
    f.close()
  return c

### Pruebas

- Función que transforma el excel en una matriz (Paso previo)

In [ ]:
rows = xlsx2matriz('us-040-2-1.xlsx')
print(np.array(rows))

[['Species' 'Wildlife Criterion (pg/L)' 'Wildlife Criterion (pg/L)']
 ['Species' 'GLWQI' 'Mercury Study Report to Congress']
 ['Mink' '2880' '1038']
 ['Otter' '1930' '764']
 ['Kingfisher' '1040' '598']
 ['Osprey' 'Not done' '1498']
 ['Eagle' '1920' '1818']]


- Función que transforma la matriz a código latex

In [ ]:
c = matriz2latex(rows)

\begin{table}
\begin{center}
\begin{tabular}{|c|c|c|}
\hline
\multirow{2}{*}{Species} & \multicolumn{2}{|c|}{Wildlife Criterion (pg/L)} \\
\cline{2-3}
 & GLWQI & Mercury Study Report to Congress \\
\hline
Mink & 2880 & 1038 \\
\hline
Otter & 1930 & 764 \\
\hline
Kingfisher & 1040 & 598 \\
\hline
Osprey & Not done & 1498 \\
\hline
Eagle & 1920 & 1818 \\
\hline
\end{tabular}
\end{center}
\end{table}


- **Función que transforma el excel en código latex** (combinación de las dos anteriores). Almacena el código latex de **tabla.xlsx** en el fichero **tabla.txt**

In [ ]:
c = xlsx2latex('us-040-2-1.xlsx')

\begin{table}
\begin{center}
\begin{tabular}{|c|c|c|}
\hline
\multirow{2}{*}{Species} & \multicolumn{2}{|c|}{Wildlife Criterion (pg/L)} \\
\cline{2-3}
 & GLWQI & Mercury Study Report to Congress \\
\hline
Mink & 2880 & 1038 \\
\hline
Otter & 1930 & 764 \\
\hline
Kingfisher & 1040 & 598 \\
\hline
Osprey & Not done & 1498 \\
\hline
Eagle & 1920 & 1818 \\
\hline
\end{tabular}
\end{center}
\end{table}


## Calculando algunas predicciones del dataset de tablas

In [ ]:
%%capture
!unzip celldataset.zip

In [ ]:
%%capture
!unzip Files_Online2PDF.zip # Predicciones de la página web

Archive:  Files_Online2PDF.zip
Created by Online2PDF.com
  inflating: eu-001-1-2.xlsx         
  inflating: eu-003-1-2.xlsx         
  inflating: eu-003-1-3.xlsx         
  inflating: eu-004-11-10.xlsx       
  inflating: eu-016-3-1.xlsx         
  inflating: eu-027-3-1.xlsx         
  inflating: us-002-3-2.xlsx         
  inflating: us-011a-3-2.xlsx        
  inflating: us-018-3-3.xlsx         
  inflating: us-024-2-1.xlsx         
  inflating: us-029-2-1.xlsx         


In [ ]:
c= xlsx2latex('eu-001-1-2.xlsx')    
print('\n \n')
c= xlsx2latex('eu-003-1-2.xlsx')   
print('\n \n') 
c= xlsx2latex('eu-003-1-3.xlsx')  
print('\n \n')
c= xlsx2latex('eu-004-11-10.xlsx')  
print('\n \n')
c= xlsx2latex('eu-016-3-1.xlsx')    
print('\n \n')
c= xlsx2latex('eu-027-3-1.xlsx')    
print('\n \n')
c= xlsx2latex('us-002-3-2.xlsx')   
print('\n \n')
c= xlsx2latex('us-011a-3-2.xlsx') 
print('\n \n')
c= xlsx2latex('us-018-3-3.xlsx')
print('\n \n')
c= xlsx2latex('us-024-2-1.xlsx') 
print('\n \n')
c= xlsx2latex('us-029-2-1.xlsx')

\begin{table}
\begin{center}
\begin{tabular}{|c|c|c|c|}
\hline
\multirow{2}{*}{} & \multicolumn{3}{|c|}{THRESHOLD FOR RELEASES} \\
\cline{2-4}
 & to air ka/year & to water ka/year & to land ka/year \\
\hline
Ammonia (NH3) & 10 000 & \multicolumn{2}{|c|}{-} \\
\hline
Carbon monoxide (CO) & 500 000 & \multicolumn{2}{|c|}{-} \\
\hline
Chlorine and inorganic compounds (as HCI) & 10 000 & \multicolumn{2}{|c|}{-} \\
\hline
Chlorofluorocarbons (CFCs) & 1 & \multicolumn{2}{|c|}{-} \\
\hline
Flourine and inorganic compounds (as HF) & 5 000 & \multicolumn{2}{|c|}{-} \\
\hline
HaIons & 1 & \multicolumn{2}{|c|}{-} \\
\hline
Hydrochlorofluorocarbons (HCFCs) & 1 & \multicolumn{2}{|c|}{-} \\
\hline
Hydrogen Cyanide (HCN) & 200 & \multicolumn{2}{|c|}{-} \\
\hline
Nitrogen oxides (NOx/ N02) & 100 000 & \multicolumn{2}{|c|}{-} \\
\hline
Non-methane volatile organic compounds (NMVOC) & 100 000 & \multicolumn{2}{|c|}{-} \\
\hline
Sulphur oxides (S0x/ S02) & 150 000 & \multicolumn{2}{|c|}{-} \\
\hline
\end

## **Más predicciones**

In [ ]:
%cd Tablas

/content/Tablas


In [ ]:
%%capture
!unzip Files_Online2PDF.zip

In [ ]:
for i in range(1,10):
  c= xlsx2latex('tabla'+str(i)+'.xlsx')    
  print('\n \n')

\begin{table}
\begin{center}
\begin{tabular}{|c|c|c|c|}
\hline
\multirow{2}{*}{} & \multicolumn{3}{|c|}{THRESHOLD FOR RELEASES} \\
\cline{2-4}
 & to air ka/year & to water ka/year & to land ka/year \\
\hline
1,1,1-trichloroethane & 100 & \multicolumn{2}{|c|}{-} \\
\hline
1,1,2,2-tetrachl oroetha ne & so & \multicolumn{2}{|c|}{-} \\
\hline
1,2-dichloroethane (EDC) & 1000 & \multicolumn{2}{|c|}{10} \\
\hline
Brominated diphenylethers (PBDE) & - & \multicolumn{2}{|c|}{1} \\
\hline
Chloro-alkanes,C10-C13 & - & \multicolumn{2}{|c|}{1} \\
\hline
Dichloromethane (DCM) & 1000 & \multicolumn{2}{|c|}{10} \\
\hline
Dieldrin & \multicolumn{3}{|c|}{1} \\
\hline
Haloaenated Oraanic Compounds (AOX) & - & \multicolumn{2}{|c|}{1000} \\
\hline
Hexabromobifenyl & \multicolumn{3}{|c|}{0.1} \\
\hline
Hexachlorobenzene (HCB) & 10 & \multicolumn{2}{|c|}{1} \\
\hline
Hexachlorobuta diene (HCBD) & - & \multicolumn{2}{|c|}{1} \\
\hline
PCDD+PCFD (Dioxins+furans) (as Teq) & \multicolumn{3}{|c|}{0.0001} \\
\hline